### class

In [ ]:
class FlagSet:
    def __init__(self, flag_map: dict):
        self._flag_map = flag_map
        self._bitfield = 0
        self._bitfield_length = max(1, self._bitfield.bit_length())

    @staticmethod
    def decode_flags(flag_map, bitfield):
        return [name for name, mask in flag_map.items() if bitfield & mask]

    @staticmethod
    def is_flag_set(flag_map, flag_name):
        mask = flag_map.get(flag_name, 0)
        return lambda bitfield: bool(bitfield & mask)

    @classmethod
    def from_value(cls, flag_map, bitfield):
        obj = cls(flag_map)
        obj._bitfield = bitfield
        return obj

    def __repr__(self):
        active = self.decode()
        return f"<FlagSet {self.binary()} → {active}>"

    def apply(self, *flag_names):
        for name in flag_names:
            self._bitfield |= self._flag_map[name]
        return self

    def clear(self, *flag_names):
        for name in flag_names:
            self._bitfield &= ~self._flag_map[name]
        return self

    def toggle(self, *flag_names):
        for name in flag_names:
            self._bitfield ^= self._flag_map[name]
        return self

    def is_set(self, flag_name):
        return bool(self._bitfield & self._flag_map[flag_name])

    def decode(self):
        return [name for name, mask in self._flag_map.items() if self._bitfield & mask]

    def mask(self, flag_name):
        return self._flag_map.get(flag_name, 0)

    def value(self):
        return self._bitfield

    def binary(self):
        return f"{self._bitfield:0{self._bitfield_length}b}"

    def describe(self):
        data = []
        for name, mask in sorted(self._flag_map.items()):
            data.append(
                {
                    "Flag Name": name,
                    "Mask": f"{mask:0{self._bitfield_length}b}",
                    "Value": mask,
                    "Active": self.is_set(name),
                }
            )
        return pd.DataFrame(data)

### usages

In [ ]:
flags = {
    "has_survived": 0b0001,
    "is_alone": 0b0010,
    "is_male": 0b0100,
    "age_below_18": 0b1000,
}
fs = FlagSet(flags)
fs.describe()

,Flag Name,Mask,Value,Active
0,age_below_18,1000,8,False
1,has_survived,1,1,False
2,is_alone,10,2,False
3,is_male,100,4,False


In [ ]:
fs.apply("has_survived", "is_male")

<FlagSet 101 → ['has_survived', 'is_male']>

In [ ]:
FlagSet.from_value(flags, 0b0101)

<FlagSet 101 → ['has_survived', 'is_male']>

In [ ]:
FlagSet.from_value(flags, 0b0101).decode()

['has_survived', 'is_male']

In [ ]:
FlagSet.from_value(flags, 0b0101).toggle("age_below_18")

<FlagSet 1101 → ['has_survived', 'is_male', 'age_below_18']>

In [ ]:
FlagSet.from_value(flags, 0b0101).clear("has_survived")

<FlagSet 100 → ['is_male']>